I fucked things up by trying to add the roster spot thing. I have it added to each dataframe but I ran into problems when merging it. Might need to take the time and go back through and fix it. Doing it as a range may not be the best way to do it. 

To complete:
- getting draft info
- getting end of year rosters for each team

Some fun things to do/strive for with this data. There is still a lot of api calls that we have left unexplored and they may be helpful or hold good data (not sure what to do with bound method)
 - Easy access to records
 - See what players have had the highest average draft position
     - are there players that have been drafted and on end of season rosters every year? 
 - Can look for a relation between draft pick and final record 

I think we could add roster spot columns to each roster dataframe. This would allow us to have roster_spot as the final index so that each of none of the name indeces are the final index. Also, I think we could add final standings to the super merge (shouldn't be that hard). Finally, we can try and work through the draft. If, at the end of it, we think that the df is too large, we have the smaller ones that we can merge and keep them seperated. 

Far ways off, but once we get closer to the season, we can create functions to gather and store the week to week data

In [ ]:
#https://github.com/cwendt94/espn-api

#pip install espn_api

#https://github.com/cwendt94/espn-api/blob/master/espn_api/football/constant.py    helpful labels

#https://github.com/cwendt94/espn-api/wiki/League-Class 

In [ ]:
#pip install --upgrade espn_api==0.36.0

Did the update above to get around the 403 error

There are already premade functions within the github files but they are slightly too advanced for me to understand at the moment. One that can be used for power rankings could be interesting. 

Some notes:
   - Roster is only final game roster
   - Mov seems to be point differential

In [1]:
from espn_api.football import League
import pandas as pd
import re

In [2]:
rzfl_id = 585565
rzfl_s2 = 'AEAiyHVgLIiwyUGOoK%2FnReBzIm%2B2Ysqt9Hz4%2BnBH6JyoHF4N%2F7W9tvhsIQvHNxjL4PjRA0LnY6YZVTnT%2BLdGedVndIsWtp%2F7iItg25WLks0XTFwTKfGWHur2%2Ba4HDmYI8XYyigXodtq9GJqgIOVXKSmr9aOWVYcJNuR82N9rumG1p6gsOLR6ZwwiJnoRXlBQZA2fGdfePXV4RnV%2BfZCJ%2BhaM%2FGMMMGlS20mGhTxDIntKssDa%2BrwcjLNyeIoJA6mYkjA%3D'
rzfl_swid = '{35B1B053-7D79-4DBF-B1B0-537D79DDBF60}'

In [3]:
#Dictionary for the league with each year as the key
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
rzfl = {}

for y in years:
    rzfl_key = y
    rzfl[rzfl_key] = League(league_id=rzfl_id, year=y, espn_s2=rzfl_s2 , swid=rzfl_swid, debug=False)

EVERYTHING ABOVE THIS BOX IS IN GOOD CONDITION TO THIS POINT

I think we will want to make a function that will take each rzfl[year] as an argument (not sure if that is correct nomenclature) and then do things like get all the teams, all the owners, and maybe the final standings. As we get more comfy, we can add more.

Trying to imagine if we can get the roster into the data frame.

Thinking big pictue, we will want year as the index on our df. For each year (ie index), lets start by getting the league.teams and from that team.owners. 

Step 1: Lets get all the teams from each year of the league into a dataframe. We will want year, team name (going to need to strip off some characters), team owner/owners (ie losing mikey, brothers combining). We could have the index set as owners

Step 2: Create a dataframe that holds the roster for each team. It will have columns being years and rows being the players for that year

In [4]:
#this function is from stack overflow and is used to pad the shorter dictionaries to be the same size
# use it when you get the error message that all arrays are not of the same length

def pad_dict_list(dict_list, padel):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if  ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

In [5]:
lg_years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

#going to create it as a dictionary first and then put it to a dataframe
lg_teams = {}

for year in lg_years:
    lg_year_key = year
    lg_teams[lg_year_key] = rzfl[year].teams    

pad_dict_list(lg_teams,0) #giving teh needed padding
    
lg_teams = pd.DataFrame(lg_teams) #creating the dataframe

lg_teams = lg_teams.astype(str) #making the values strings

In [6]:
#from chatgpt; used to strip down the values to just team names

# Function to extract team name from 'Team(name)' format
def extract_team_name(team_str):
    match = re.match(r'Team\((.*?)\)', team_str)
    if match:
        return match.group(1)
    else:
        return team_str

# Apply the function to each element in the DataFrame
lg_teams = lg_teams.applymap(extract_team_name)
lg_teams

,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Dez Nuts,Seattle Sea Men,Big Ballers,Big Ballers,Big Ballers,Drake's New Favorite Team,Drake's New Favorite Team,Drake's New Favorite Team,Drake's New Favorite Team
1,Kyle Smile's,Kyle Smile's,Kyle Smile's,Kyle Smile's,Kyle Smile's,Kyle Smile's,Kyle Smyles,Kyle Smyles,Kyle Smyles
2,Corn On The Cobb,Slob On My Cobb,"Wentzylvania, USA",Kittle Corn,Kittle Corn,Kittle Corn,Kittle Corn,"Wentzylvania, USA","Wentzylvania, USA"
3,Hernandez Hitmen,Jaggin Off,Jaggin Off,Jaggin Off,Jaggin Off,Jaggin Off,The Magic Gus Bus,FRANK'S BIG UGLIES,FRANK'S BIG UGLIES
4,Team Domination,Team Gonzalez,Team Gonzalez,Team Gonzalez,Team Gonzalez,The Sleepers,🎱 Ball's,🎱 Ball's,🎱 Ball's
5,Potato Chips,Potato Chips,Potato Chips,Potato Chips,Potato Chips,Potato Chips,Parkys Penguins,Country Blumpkins,Country Blumpkins
6,French Fries,Frosty The Snowman,The Mixon Hitmen,Team Farin,Team Farin,Mixon It Up,Mixon It Up,Mixon Up Some Kittle Corn,Busch-Eating Beerslingers
7,Bend it like Beckham Jr.,Show Me Your TD's,Show Me Your TD's,Prestige Worldwide,Prestige Worldwide,Prestige Worldwide,Prestige Worldwide,The Bad Bruddas,The Bad Bruddas
8,50 Shades Of Jay,The Plowers,The Plowers,The Plowers,The Plowers,The Plowers,The Plowers,The Plowers,The Plowers
9,Mean Machine,Morning Wood,Morning Wood,Morning Wood,Morning Wood,Morning Wood,Morning Wood,Morning Wood,Morning Wood


In [7]:
#creating a team_id col and setting that as index. Hoping this will help get when we need to get owners mapped on
lg_teams['team_id'] = [0,1,2,3,4,5,6,7,8,9,10,11]
lg_teams = lg_teams.set_index('team_id')

In [8]:
#renaming the columns for future merging ease
#new_columns = {col: f"{col}_name" for col in lg_teams.columns} #from gpt
#lg_teams.rename(columns=new_columns, inplace=True)
lg_teams   

,2015,2016,2017,2018,2019,2020,2021,2022,2023
team_id,,,,,,,,,
0,Dez Nuts,Seattle Sea Men,Big Ballers,Big Ballers,Big Ballers,Drake's New Favorite Team,Drake's New Favorite Team,Drake's New Favorite Team,Drake's New Favorite Team
1,Kyle Smile's,Kyle Smile's,Kyle Smile's,Kyle Smile's,Kyle Smile's,Kyle Smile's,Kyle Smyles,Kyle Smyles,Kyle Smyles
2,Corn On The Cobb,Slob On My Cobb,"Wentzylvania, USA",Kittle Corn,Kittle Corn,Kittle Corn,Kittle Corn,"Wentzylvania, USA","Wentzylvania, USA"
3,Hernandez Hitmen,Jaggin Off,Jaggin Off,Jaggin Off,Jaggin Off,Jaggin Off,The Magic Gus Bus,FRANK'S BIG UGLIES,FRANK'S BIG UGLIES
4,Team Domination,Team Gonzalez,Team Gonzalez,Team Gonzalez,Team Gonzalez,The Sleepers,🎱 Ball's,🎱 Ball's,🎱 Ball's
5,Potato Chips,Potato Chips,Potato Chips,Potato Chips,Potato Chips,Potato Chips,Parkys Penguins,Country Blumpkins,Country Blumpkins
6,French Fries,Frosty The Snowman,The Mixon Hitmen,Team Farin,Team Farin,Mixon It Up,Mixon It Up,Mixon Up Some Kittle Corn,Busch-Eating Beerslingers
7,Bend it like Beckham Jr.,Show Me Your TD's,Show Me Your TD's,Prestige Worldwide,Prestige Worldwide,Prestige Worldwide,Prestige Worldwide,The Bad Bruddas,The Bad Bruddas
8,50 Shades Of Jay,The Plowers,The Plowers,The Plowers,The Plowers,The Plowers,The Plowers,The Plowers,The Plowers


In [9]:
#stacking to get years as a variable

lg_teams_stack = lg_teams.stack()

lg_teams_stack = lg_teams_stack.reset_index()

lg_teams_stack.columns = ['team_id', 'year', 'team_name']

In [10]:
lg_teams_stack

,team_id,year,team_name
0,0,2015,Dez Nuts
1,0,2016,Seattle Sea Men
2,0,2017,Big Ballers
3,0,2018,Big Ballers
4,0,2019,Big Ballers
...,...,...,...
103,11,2019,Unknown Unknown
104,11,2020,Wenyen Footlongs
105,11,2021,Wenyen Footlongs
106,11,2022,Wenyen Footlongs


Step 1 part c: We need to figure out a way to attach owners into this dataframe. Owners are kinda a contstnt and kinda a variable. Not really sure if they will go into rows or columns. 

I think what may be giving us problems is adding to the league. It is messing up the index range

In [11]:
#going to start by just creating a dictionary to see if we can call owners correctly

lg_years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] #dont really need to define again but bringing it down anyway
team_ids = [0,1,2,3,4,5,6,7,8,9,10,11] #can shorten this once we know things are working


owners = {}

#from gpt
for year in lg_years:
    num_teams = 10 if year == 2015 else 12  # Determine the number of teams for each year
    owners[year] = {}  # Create a dictionary for the current year
    
    for team_id in range(num_teams):
        team_owners = []  # List to store owners for the current team
        
        # Iterate through owners for the current team
        for owner in rzfl[year].teams[team_id].owners:
            team_owners.append(owner['firstName'])
        
        owners[year][team_id] = team_owners

owners = pd.DataFrame(owners) #getting it into dataframe

#not sure if we really need them in strings. We will find out as we try to edit and strip them down
owners = owners.astype(str) #getting them into strings so that they are easier to manipulate

In [12]:
#removing the unneeded brackets

#from gpt
for column in owners.columns:
    owners[column] = owners[column].apply(lambda x: re.sub(r"\[|\]|'", "", str(x)))

Lets strip the values and then go in and add

Noticing that we might need to go in and edit some of the owners for years when a club had 2 owners. For example, team 7 should have parky and trevor as owners in 2023. Also, team 5 is no longer ran by mikey. This will take some fixing 

Here are the fixes we need:
 - check team 4 (was it really brock or brandon in the league??)
 - edit team 5 (some years mario, but are some also parky?)
 - edit name for team 6 for consistency
 - for team 7, show that there are coowners for certain years (2023 for sure)
 - edit name for team 10 for consistence
 - I think we should be good with the two nans in 2015
 
 
- For team 4: might acutally be correct, 2015 may be brandon
- for team 5: 2017, 2018, 2019, 2020 - parky and mikey; 2021 - parky; 2022, 2023 - Mario
- for team 7: 2022, 2023 - parky and trevor

In [13]:
#going in to manually fix the owners, getting everything to have first letter capitals

#fixing team 1
for year in lg_years:
    owners[year][1] = 'Kyle'

#fixing team 3
for year in lg_years:
    owners[year][3] ='Jack'

#fixing team 5
for year in 2015, 2016: #getting them all to mikey first
    owners[year][5] = 'Mikey'
for year in 2017, 2018, 2019, 2020:
    owners[year][5] = 'Mikey, Parker'
owners[2021][5] = 'Parker'
for year in 2022, 2023:
    owners[year][5] = 'Mario'
    
#fixing team 6
for year in lg_years:
    owners[year][6] = 'Chris'
    
#fixing team 7
owners[2022][7] = 'Trevor, Parker'
owners[2023][7] ='Trevor, Parker'

#fixing team 10
for year in 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023:
    owners[year][10] = 'Eli'

In [14]:
#creating team_id column
owners['team_id'] = [0,1,2,3,4,5,6,7,8,9,10,11]
owners = owners.set_index('team_id')
owners

,2015,2016,2017,2018,2019,2020,2021,2022,2023
team_id,,,,,,,,,
0,Sam,Sam,Sam,Sam,Sam,Sam,Sam,Sam,Sam
1,Kyle,Kyle,Kyle,Kyle,Kyle,Kyle,Kyle,Kyle,Kyle
2,Hunter,Hunter,Hunter,Hunter,Hunter,Hunter,Hunter,Hunter,Hunter
3,Jack,Jack,Jack,Jack,Jack,Jack,Jack,Jack,Jack
4,Brock,Andrew,Andrew,Andrew,Andrew,Andrew,Andrew,Andrew,Andrew
5,Mikey,Mikey,"Mikey, Parker","Mikey, Parker","Mikey, Parker","Mikey, Parker",Parker,Mario,Mario
6,Chris,Chris,Chris,Chris,Chris,Chris,Chris,Chris,Chris
7,Trevor,Trevor,Trevor,Trevor,Trevor,Trevor,Trevor,"Trevor, Parker","Trevor, Parker"
8,Zach,Zach,Zach,Zach,Zach,Zach,Zach,Zach,Zach


In [15]:
#renaming the columns for future merging ease
#new_columns = {col: f"{col}_owner" for col in owners.columns} #from gpt
#owners.rename(columns=new_columns, inplace=True)
owners  

,2015,2016,2017,2018,2019,2020,2021,2022,2023
team_id,,,,,,,,,
0,Sam,Sam,Sam,Sam,Sam,Sam,Sam,Sam,Sam
1,Kyle,Kyle,Kyle,Kyle,Kyle,Kyle,Kyle,Kyle,Kyle
2,Hunter,Hunter,Hunter,Hunter,Hunter,Hunter,Hunter,Hunter,Hunter
3,Jack,Jack,Jack,Jack,Jack,Jack,Jack,Jack,Jack
4,Brock,Andrew,Andrew,Andrew,Andrew,Andrew,Andrew,Andrew,Andrew
5,Mikey,Mikey,"Mikey, Parker","Mikey, Parker","Mikey, Parker","Mikey, Parker",Parker,Mario,Mario
6,Chris,Chris,Chris,Chris,Chris,Chris,Chris,Chris,Chris
7,Trevor,Trevor,Trevor,Trevor,Trevor,Trevor,Trevor,"Trevor, Parker","Trevor, Parker"
8,Zach,Zach,Zach,Zach,Zach,Zach,Zach,Zach,Zach


In [16]:
#lets stack owners

#stacking to get years as a variable

owners_stack = owners.stack()

owners_stack = owners_stack.reset_index()

owners_stack.columns = ['team_id', 'year', 'owner']

owners_stack = owners_stack.drop(0)

owners_stack

,team_id,year,owner
1,0,2016,Sam
2,0,2017,Sam
3,0,2018,Sam
4,0,2019,Sam
5,0,2020,Sam
...,...,...,...
103,11,2019,Brandon
104,11,2020,Brandon
105,11,2021,Brandon
106,11,2022,Brandon


At this point, the owners df is all set

In [17]:
#creating a merged df of owners and teams
lg_teams = lg_teams_stack.reset_index()
owners = owners_stack.reset_index()
clubs = pd.merge(lg_teams_stack, owners_stack, how='inner', on=['team_id', 'year'])

I think we would want to add in final standings to clubs if we add it anywhere

In [18]:
standings = {}

for year in lg_years:
    num_teams = 10 if year == 2015 else 12  # Determine the number of teams for each year
    standings[year] = {}  # Create a dictionary for the current year
    
    for team_id in range(num_teams):
        final_standing = rzfl[year].teams[team_id].final_standing
        standings[year][team_id] = final_standing

# Convert dictionary to DataFrame
standings = pd.DataFrame(standings)

standings['team_id'] = [0,1,2,3,4,5,6,7,8,9,10,11]

standings

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id
0,3.0,6,1,7,6,6,2,3,10,0
1,8.0,7,12,9,10,11,9,10,3,1
2,2.0,4,2,4,8,1,6,8,1,2
3,4.0,1,4,6,1,10,3,2,2,3
4,5.0,5,7,2,5,4,10,4,11,4
5,10.0,10,9,1,11,3,4,7,9,5
6,6.0,9,6,11,2,5,7,9,8,6
7,1.0,3,8,12,7,8,5,6,6,7
8,7.0,12,11,8,4,9,8,11,7,8
9,9.0,2,10,10,9,2,11,12,4,9


In [19]:
#stacking to get years as a variable

standings = standings.set_index('team_id')

standings_stack = standings.stack()

standings_stack = standings_stack.reset_index()

standings_stack.columns = ['team_id', 'year', 'final_standing']

standings_stack

,team_id,year,final_standing
0,0,2015,3.0
1,0,2016,6.0
2,0,2017,1.0
3,0,2018,7.0
4,0,2019,6.0
...,...,...,...
101,11,2019,3.0
102,11,2020,7.0
103,11,2021,1.0
104,11,2022,5.0


In [20]:
clubs_standings = pd.merge(clubs, standings_stack, how='outer', on=['team_id', 'year'])
clubs_standings = clubs_standings.drop(clubs_standings.index[-1])
clubs_standings

,team_id,year,team_name,owner,final_standing
0,0,2016,Seattle Sea Men,Sam,6.0
1,0,2017,Big Ballers,Sam,1.0
2,0,2018,Big Ballers,Sam,7.0
3,0,2019,Big Ballers,Sam,6.0
4,0,2020,Drake's New Favorite Team,Sam,6.0
...,...,...,...,...,...
102,11,2019,Unknown Unknown,Brandon,3.0
103,11,2020,Wenyen Footlongs,Brandon,7.0
104,11,2021,Wenyen Footlongs,Brandon,1.0
105,11,2022,Wenyen Footlongs,Brandon,5.0


# clubs now has the variables of year, team_id, team_name, and owner all in the columns. 

In [21]:
#creating a dictionary of team1 roster
team1_id = 0
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team1_rost = {}

for y in years: 
    team1_rost_key = y
    team1_rost[team1_rost_key] = rzfl[y].teams[team1_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team1_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team1_ros_df = pd.DataFrame(team1_rost[2015])
team1_ros_df = team1_ros_df.rename(columns={0: 2015})

for y in needs:
    team1_ros_df[y] = team1_rost[y]

team1_ros_df = team1_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team1_ros_df.columns:
    team1_ros_df[column] = team1_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team1_ros_df['team_id'] = team1_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    
    
team1_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17] #made to match the index 
    
team1_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,Dez Bryant,Lamar Miller,Todd Gurley II,Odell Beckham Jr.,David Johnson,Aaron Jones,Kirk Cousins,Stefon Diggs,Stefon Diggs,0,1
1,LeSean McCoy,Alshon Jeffery,Alshon Jeffery,Travis Kelce,Mike Evans,Adam Thielen,Kareem Hunt,Travis Kelce,Darren Waller,0,2


In [22]:
#creating a dictionary of team2 roster (all in one cell)
team2_id = 1 #CHANGE THIS
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team2_rost = {}

for y in years: 
    team2_rost_key = y
    team2_rost[team2_rost_key] = rzfl[y].teams[team2_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team2_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team2_ros_df = pd.DataFrame(team2_rost[2015])
team2_ros_df = team2_ros_df.rename(columns={0: 2015})

for y in needs:
    team2_ros_df[y] = team2_rost[y]

team2_ros_df = team2_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team2_ros_df.columns:
    team2_ros_df[column] = team2_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team2_ros_df['team_id'] = team2_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    
    
team2_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]      
    
team2_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,Aaron Rodgers,Todd Gurley,Amari Cooper,Saquon Barkley,Ezekiel Elliott,Mike Evans,Diontae Johnson,Davante Adams,Davante Adams,1,1
1,Greg Olsen,Amari Cooper,Christian McCaffrey,A.J. Green,Derrick Henry,Melvin Gordon III,Kenyan Drake,Leonard Fournette,Derrick Henry,1,2


In [23]:
#creating a dictionary of team2 roster (all in one cell)
team3_id = 2 #CHANGE THIS
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team3_rost = {}

for y in years: 
    team3_rost_key = y
    team3_rost[team3_rost_key] = rzfl[y].teams[team3_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team3_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team3_ros_df = pd.DataFrame(team3_rost[2015])
team3_ros_df = team3_ros_df.rename(columns={0: 2015})

for y in needs:
    team3_ros_df[y] = team3_rost[y]

team3_ros_df = team3_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team3_ros_df.columns:
    team3_ros_df[column] = team3_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team3_ros_df['team_id'] = team3_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    
    
team3_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]      
    
team3_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,A.J. Green,David Johnson,Melvin Gordon,Dalvin Cook,Todd Gurley II,Tyreek Hill,Aaron Jones,Dalvin Cook,Christian McCaffrey,2,1
1,Drew Brees,Brandin Cooks,Ezekiel Elliott,Larry Fitzgerald,Sony Michel,Kenny Golladay,Mike Evans,Nick Chubb,Cooper Kupp,2,2


In [24]:
#creating a dictionary of team4 roster
team4_id = 3
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team4_rost = {}

for y in years: 
    team4_rost_key = y
    team4_rost[team4_rost_key] = rzfl[y].teams[team4_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team4_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team4_ros_df = pd.DataFrame(team4_rost[2015])
team4_ros_df = team4_ros_df.rename(columns={0: 2015})

for y in needs:
    team4_ros_df[y] = team4_rost[y]

team4_ros_df = team4_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team4_ros_df.columns:
    team4_ros_df[column] = team4_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team4_ros_df['team_id'] = team4_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    
    
team4_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]     
    
team4_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,Andrew Luck,Cam Newton,Devonta Freeman,Julio Jones,DeAndre Hopkins,Miles Sanders,Alvin Kamara,Ja'Marr Chase,Tony Pollard,3,1
1,Calvin Johnson,Mike Evans,Isaiah Crowell,Keenan Allen,Travis Kelce,Calvin Ridley,Josh Allen,A.J. Brown,A.J. Brown,3,2


In [25]:
#creating a dictionary of team5 roster
team5_id = 4
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team5_rost = {}

for y in years: 
    team5_rost_key = y
    team5_rost[team5_rost_key] = rzfl[y].teams[team5_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team5_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team5_ros_df = pd.DataFrame(team5_rost[2015])
team5_ros_df = team5_ros_df.rename(columns={0: 2015})

for y in needs:
    team5_ros_df[y] = team5_rost[y]

team5_ros_df = team5_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team5_ros_df.columns:
    team5_ros_df[column] = team5_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team5_ros_df['team_id'] = team5_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    
    
team5_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]  
    
team5_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,Eddie Lacy,Julio Jones,Michael Thomas,Alvin Kamara,Julio Jones,Dalvin Cook,Saquon Barkley,Najee Harris,Garrett Wilson,4,1
1,Adrian Peterson,Jordan Reed,C.J. Anderson,Demaryius Thomas,Michael Thomas,Julio Jones,DeAndre Hopkins,Tyreek Hill,Aaron Jones,4,2


In [26]:
#creating a dictionary of team6 roster
team6_id = 5
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team6_rost = {}

for y in years: 
    team6_rost_key = y
    team6_rost[team6_rost_key] = rzfl[y].teams[team6_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team6_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team6_ros_df = pd.DataFrame(team6_rost[2015])
team6_ros_df = team6_ros_df.rename(columns={0: 2015})

for y in needs:
    team6_ros_df[y] = team6_rost[y]

team6_ros_df = team6_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team6_ros_df.columns:
    team6_ros_df[column] = team6_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team6_ros_df['team_id'] = team6_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    

team6_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]      

team6_ros_df.head(2)


,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,Marshawn Lynch,Jarvis Landry,Antonio Brown,Antonio Brown,JuJu Smith-Schuster,JuJu Smith-Schuster,Austin Ekeler,Joe Mixon,Austin Ekeler,5,1
1,Russell Wilson,Carson Palmer,Leonard Fournette,Jordan Howard,James Conner,James Conner,Najee Harris,Mike Williams,DK Metcalf,5,2


In [27]:
#creating a dictionary of team7 roster
team7_id = 6
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team7_rost = {}

for y in years: 
    team7_rost_key = y
    team7_rost[team7_rost_key] = rzfl[y].teams[team7_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team7_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team7_ros_df = pd.DataFrame(team7_rost[2015])
team7_ros_df = team7_ros_df.rename(columns={0: 2015})

for y in needs:
    team7_ros_df[y] = team7_rost[y]

team7_ros_df = team7_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team7_ros_df.columns:
    team7_ros_df[column] = team7_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team7_ros_df['team_id'] = team7_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    
 
team7_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]      
    
team7_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,Le'Veon Bell,Brandon Marshall,Mark Ingram,Leonard Fournette,Alvin Kamara,Joe Mixon,Travis Kelce,Christian McCaffrey,Chris Olave,6,1
1,Demaryius Thomas,Andrew Luck,Larry Fitzgerald,Christian McCaffrey,Julian Edelman,Davante Adams,Joe Mixon,Michael Pittman Jr.,Patrick Mahomes,6,2


In [28]:
#creating a dictionary of team8 roster
team8_id = 7
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team8_rost = {}

for y in years: 
    team8_rost_key = y
    team8_rost[team8_rost_key] = rzfl[y].teams[team8_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team8_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team8_ros_df = pd.DataFrame(team8_rost[2015])
team8_ros_df = team8_ros_df.rename(columns={0: 2015})

for y in needs:
    team8_ros_df[y] = team8_rost[y]

team8_ros_df = team8_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team8_ros_df.columns:
    team8_ros_df[column] = team8_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team8_ros_df['team_id'] = team8_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    

team8_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]      
    
team8_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,Odell Beckham Jr.,T.Y. Hilton,Davante Adams,LeSean McCoy,Saquon Barkley,T.J. Hockenson,Tyreek Hill,JuJu Smith-Schuster,Jaylen Waddle,7,1
1,Rob Gronkowski,Vikings D/ST,Stefon Diggs,Tyreek Hill,Josh Jacobs,Justin Herbert,David Montgomery,Curtis Samuel,Najee Harris,7,2


In [29]:
#creating a dictionary of team8 roster
team9_id = 8
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team9_rost = {}

for y in years: 
    team9_rost_key = y
    team9_rost[team9_rost_key] = rzfl[y].teams[team9_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team9_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team9_ros_df = pd.DataFrame(team9_rost[2015])
team9_ros_df = team9_ros_df.rename(columns={0: 2015})

for y in needs:
    team9_ros_df[y] = team9_rost[y]

team9_ros_df = team9_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team9_ros_df.columns:
    team9_ros_df[column] = team9_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team9_ros_df['team_id'] = team9_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    

team9_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]      

team9_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,Antonio Brown,Adrian Peterson,David Johnson,Mike Evans,Le'Veon Bell,Kenyan Drake,Ezekiel Elliott,Derrick Henry,Bijan Robinson,8,1
1,Matt Forte,Julian Edelman,Lamar Miller,Derrick Henry,Dalvin Cook,Austin Ekeler,Keenan Allen,CeeDee Lamb,Amon-Ra St. Brown,8,2


In [30]:
#creating a dictionary of team10 roster
team10_id = 9
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team10_rost = {}

for y in years: 
    team10_rost_key = y
    team10_rost[team10_rost_key] = rzfl[y].teams[team10_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team10_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team10_ros_df = pd.DataFrame(team10_rost[2015])
team10_ros_df = team10_ros_df.rename(columns={0: 2015})

for y in needs:
    team10_ros_df[y] = team10_rost[y]

team10_ros_df = team10_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team10_ros_df.columns:
    team10_ros_df[column] = team10_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team10_ros_df['team_id'] = team10_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    

team10_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]      

team10_ros_df.head(2)

,2015,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,C.J. Anderson,LeSean McCoy,Dez Bryant,Todd Gurley II,Christian McCaffrey,Travis Kelce,Corey Davis,Joe Burrow,Tyreek Hill,9,1
1,Matt Ryan,Aaron Rodgers,Jordan Howard,Rob Gronkowski,Keenan Allen,Patrick Mahomes,Joe Burrow,Allen Lazard,Jalen Hurts,9,2


In [31]:
#teams 11 and 12 may give us trouble because they were not in the league the first year

#creating a dictionary of team11 roster
team11_id = 10
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team11_rost = {}

for y in years: 
    team11_rost_key = y
    team11_rost[team11_rost_key] = rzfl[y].teams[team11_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team11_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team11_ros_df = pd.DataFrame(team11_rost[2016])
team11_ros_df = team11_ros_df.rename(columns={0: 2016})

for y in needs:
    team11_ros_df[y] = team11_rost[y]

team11_ros_df = team11_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team11_ros_df.columns:
    team11_ros_df[column] = team11_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team11_ros_df['team_id'] = team11_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    

team11_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]      

team11_ros_df.head(2)

,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,A.J. Green,Le'Veon Bell,Kareem Hunt,Tyreek Hill,Christian McCaffrey,Hunter Renfrow,Austin Ekeler,Dameon Pierce,10,1
1,Justin Forsett,Brandin Cooks,Michael Thomas,Odell Beckham Jr.,Lamar Jackson,Rams D/ST,Saquon Barkley,Amari Cooper,10,2


In [32]:
#teams 11 and 12 may give us trouble because they were not in the league the first year

#creating a dictionary of team11 roster
team12_id = 11
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

team12_rost = {}

for y in years: 
    team12_rost_key = y
    team12_rost[team12_rost_key] = rzfl[y].teams[team12_id].roster
    
#since some years do not have the same amount of roster spots, I think we need to give it padding again (luckily we have pad_func)

pad_dict_list(team12_rost,0) #giving it the padding

#creating a for loop to add to the dataframe
needs = [2017, 2018, 2019, 2020, 2021, 2022, 2023] 

team12_ros_df = pd.DataFrame(team12_rost[2016])
team12_ros_df = team12_ros_df.rename(columns={0: 2016})

for y in needs:
    team12_ros_df[y] = team12_rost[y]

team12_ros_df = team12_ros_df.astype(str) #getting them all to be strings so we can strip them

#lets strip off the player names (from gpt)
pattern = r'Player\((.*?)\)'

for column in team12_ros_df.columns:
    team12_ros_df[column] = team12_ros_df[column].apply(lambda x: re.sub(pattern, r'\1', x) if isinstance(x, str) else x)

team12_ros_df['team_id'] = team12_id #adding in an id column; could also add a roster spot column but not sure what exaclty that achieves    

team12_ros_df['roster_spot'] = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]      

team12_ros_df.head(2)

,2016,2017,2018,2019,2020,2021,2022,2023,team_id,roster_spot
0,DeAndre Hopkins,Julio Jones,David Johnson,Davante Adams,Nick Chubb,Derrick Henry,Jonathan Taylor,Justin Jefferson,11,1
1,Allen Robinson,Rob Gronkowski,Davante Adams,Nick Chubb,DeAndre Hopkins,DK Metcalf,Mark Andrews,Travis Etienne Jr.,11,2


# We have all rosters in their own dataframe and I will try to do the stacking when we have one large df

Going to merge all the rosters into one dataframe. This will help us see what players are most often on final day rosters

In [33]:
def merger(df1, df2):
    return pd.merge(df1, df2, how='outer', on=['team_id', 'roster_spot', 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [34]:
#merging all the team rosters (THERE has got to be a for loop for this. In fact this whole kernel 
#  could be an exercise in finding more concise code)

#going to need to be slgihtly different for team 11 and 12 cuz they don't have 2015 data

#teams 1 and 2
super_rosters = merger(team1_ros_df, team2_ros_df)

#team 3 
super_rosters = merger(super_rosters, team3_ros_df)

#team 4 
super_rosters = merger(super_rosters, team4_ros_df)

#team 5 
super_rosters = merger(super_rosters, team5_ros_df)

#team 6 
super_rosters = merger(super_rosters, team6_ros_df)

#team 7 
super_rosters = merger(super_rosters, team7_ros_df)

#team 8 
super_rosters = merger(super_rosters, team8_ros_df)

#team 9 
super_rosters = merger(super_rosters, team9_ros_df)

#team 10 
super_rosters = merger(super_rosters, team10_ros_df)

#team 11 (no longer has 2015)
super_rosters = pd.merge(super_rosters, team11_ros_df, how='outer', on=['team_id', 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

#team 12 (no longer has 2015)
super_rosters = pd.merge(super_rosters, team12_ros_df, how='outer', on=['team_id', 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [35]:
#from gpt
# Combine the values from 'roster_spot_x', 'roster_spot_y', and 'roster_spot'
super_rosters['roster_spot'] = super_rosters['roster_spot_x'].combine_first(super_rosters['roster_spot_y']).combine_first(super_rosters['roster_spot'])

# Drop 'roster_spot_x' and 'roster_spot_y' columns
super_rosters.drop(columns=['roster_spot_x', 'roster_spot_y'], inplace=True)


We now have all the roster with team_id on one dataframe but lets stack it properly. We could probably create a stacking function to run where teh only argument is the dataframe and maybe the columns you want.

In [36]:
roster_stack = super_rosters.set_index('team_id').stack()

roster_stack = roster_stack.reset_index()

roster_stack.columns = ['team_id', 'year', 'roster_spot']

roster_stack = roster_stack.drop(roster_stack.index[-1])

roster_stack

,team_id,year,roster_spot
0,0,2015,Dez Bryant
1,0,2016,Lamar Miller
2,0,2017,Todd Gurley II
3,0,2018,Odell Beckham Jr.
4,0,2019,David Johnson
...,...,...,...
1970,11,2019,0
1971,11,2020,0
1972,11,2021,Trey Lance
1973,11,2022,0


In [37]:
super_merge = pd.merge(roster_stack, clubs_standings, how='inner', on=['team_id', 'year'])
super_merge

,team_id,year,roster_spot,team_name,owner,final_standing
0,0,2016,Lamar Miller,Seattle Sea Men,Sam,6.0
1,0,2016,Alshon Jeffery,Seattle Sea Men,Sam,6.0
2,0,2016,Coby Fleener,Seattle Sea Men,Sam,6.0
3,0,2016,Panthers D/ST,Seattle Sea Men,Sam,6.0
4,0,2016,Justin Tucker,Seattle Sea Men,Sam,6.0
...,...,...,...,...,...,...
1753,11,2023,Keaton Mitchell,Wenyen Footlongs,Brandon,5.0
1754,11,2023,Pat Freiermuth,Wenyen Footlongs,Brandon,5.0
1755,11,2023,D'Ernest Johnson,Wenyen Footlongs,Brandon,5.0
1756,11,2023,Isaiah Likely,Wenyen Footlongs,Brandon,5.0


In [38]:
super_merge = super_merge.set_index(['team_id', 'owner', 'team_name', 'year', 'final_standing'])
super_merge

roster_spot
team_id owner   team_name        year final_standing                  
0       Sam     Seattle Sea Men  2016 6.0                 Lamar Miller
                                      6.0               Alshon Jeffery
                                      6.0                 Coby Fleener
                                      6.0                Panthers D/ST
                                      6.0                Justin Tucker
...                                                                ...
11      Brandon Wenyen Footlongs 2023 5.0              Keaton Mitchell
                                      5.0               Pat Freiermuth
                                      5.0             D'Ernest Johnson
                                      5.0                Isaiah Likely
                                      5.0                            0

[1758 rows x 1 columns]

The index isn't turning out exaclt as I thought it would but that is okay. Also, roster spot getting unstacked changed how I wanted it to look but its not that bad. 

In [ ]:
#getting it output to an excel file
league_rosters_full = super_merge 

league_rosters_full.to_excel('league_roster_full.xlsx', index=True)

Doing some playing to see if we can find a player that has won on multiple teams

In [39]:
super_merge = super_merge.reset_index()

In [40]:
super_merge

,team_id,owner,team_name,year,final_standing,roster_spot
0,0,Sam,Seattle Sea Men,2016,6.0,Lamar Miller
1,0,Sam,Seattle Sea Men,2016,6.0,Alshon Jeffery
2,0,Sam,Seattle Sea Men,2016,6.0,Coby Fleener
3,0,Sam,Seattle Sea Men,2016,6.0,Panthers D/ST
4,0,Sam,Seattle Sea Men,2016,6.0,Justin Tucker
...,...,...,...,...,...,...
1753,11,Brandon,Wenyen Footlongs,2023,5.0,Keaton Mitchell
1754,11,Brandon,Wenyen Footlongs,2023,5.0,Pat Freiermuth
1755,11,Brandon,Wenyen Footlongs,2023,5.0,D'Ernest Johnson
1756,11,Brandon,Wenyen Footlongs,2023,5.0,Isaiah Likely


In [42]:
champions = super_merge[super_merge['final_standing']==1.0]
champions

,team_id,owner,team_name,year,final_standing,roster_spot
17,0,Sam,Big Ballers,2017,1.0,Todd Gurley II
18,0,Sam,Big Ballers,2017,1.0,Alshon Jeffery
19,0,Sam,Big Ballers,2017,1.0,Travis Kelce
20,0,Sam,Big Ballers,2017,1.0,Drew Brees
21,0,Sam,Big Ballers,2017,1.0,Martavis Bryant
...,...,...,...,...,...,...
1719,11,Brandon,Wenyen Footlongs,2021,1.0,Sony Michel
1720,11,Brandon,Wenyen Footlongs,2021,1.0,Gabriel Davis
1721,11,Brandon,Wenyen Footlongs,2021,1.0,Jaret Patterson
1722,11,Brandon,Wenyen Footlongs,2021,1.0,Tyler Huntley


In [50]:
# Count occurrences of each player name
player_counts = champions['roster_spot'].value_counts()

# Filter for player names with more than one occurrence
repeat_players = player_counts[player_counts > 1]

# Display repeat player names and their counts
print("Repeat Player Names:")
print(repeat_players)

Repeat Player Names:
Chargers D/ST        3
Travis Kelce         3
Cooper Kupp          3
0                    3
Sony Michel          2
Leonard Fournette    2
Derrick Henry        2
Sammy Watkins        2
Frank Gore           2
Cardinals D/ST       2
Austin Ekeler        2
Antonio Gates        2
Gabriel Davis        2
Mike Evans           2
DeAndre Hopkins      2
Alvin Kamara         2
Martavis Bryant      2
Name: roster_spot, dtype: int64
